In [1]:
from sqlalchemy import create_engine
import os
import pandas as pd
from dotenv import load_dotenv

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)

def create_connection():

    load_dotenv()
    host = os.environ.get('DB_DESTINATION_HOST')
    port = os.environ.get('DB_DESTINATION_PORT')
    db = os.environ.get('DB_DESTINATION_NAME')
    username = os.environ.get('DB_DESTINATION_USER')
    password = os.environ.get('DB_DESTINATION_PASSWORD')

    conn = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db}')
    return conn

# установите соединение с базой
conn = create_connection()

In [6]:
from sqlalchemy import inspect

inspector = inspect(conn)
if 'clean_flats_churn' in inspector.get_table_names():
    # flats_table.drop(engine)
    print(True)
else:
    print(False)
# flats_table.create(engine)

False


In [8]:
conn.execute("ALTER TABLE clean_flats_churn DROP CONSTRAINT IF EXISTS unique_flat_constraint")
        

/tmp/ipykernel_436529/1001502051.py:1: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conn.execute("ALTER TABLE clean_flats_churn DROP CONSTRAINT IF EXISTS unique_flat_constraint")


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "clean_flats_churn" does not exist

[SQL: ALTER TABLE clean_flats_churn DROP CONSTRAINT IF EXISTS unique_flat_constraint]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [7]:
sql = '''select f.* 
    from clean_flats_churn as f
    join buildings as b on f.building_id=b.id'''

test = pd.read_sql(sql, conn)
test

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "clean_flats_churn" does not exist
LINE 2:     from clean_flats_churn as f
                 ^

[SQL: select f.* 
    from clean_flats_churn as f
    join buildings as b on f.building_id=b.id]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [29]:
data = pd.read_sql(
    '''select f.id,f.floor,f.is_apartment,f.kitchen_area,f.living_area,f.rooms,
    f.studio,f.total_area,f.price,f.building_id,
    b.build_year,b.building_type_int,b.latitude,b.longitude,
    b.ceiling_height,b.flats_count,b.floors_total,b.has_elevator 
    from flats as f
    join buildings as b on f.building_id=b.id''', conn)
data

,id,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9,False,9.90,19.900000,1,False,35.099998,9500000,6220,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,7,False,0.00,16.600000,1,False,43.000000,13500000,18012,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,9,False,9.00,32.000000,2,False,56.000000,13500000,17821,2000,4,55.740040,37.761742,2.70,80,10,True
3,3,1,False,10.10,43.099998,3,False,76.000000,20000000,18579,2002,4,55.672016,37.570877,2.64,771,17,True
4,4,3,False,3.00,14.000000,1,False,24.000000,5200000,9293,1971,1,55.808807,37.707306,2.60,208,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141357,141357,16,False,11.00,18.000000,1,False,42.000000,10500000,22455,2013,4,55.626579,37.313503,2.64,672,25,True
141358,141358,5,False,5.28,28.330000,2,False,41.110001,7400000,3162,1960,1,55.727470,37.768677,2.48,80,5,False
141359,141359,7,False,5.30,20.000000,1,False,31.500000,9700000,6513,1966,4,55.704315,37.506584,2.64,72,9,True
141360,141360,15,False,13.80,33.700001,2,False,65.300003,11750000,23952,2017,4,55.699863,37.939564,2.70,480,25,True


In [30]:
buff=data.copy()

In [31]:
buff.dtypes

id                     int64
floor                  int64
is_apartment            bool
kitchen_area         float64
living_area          float64
rooms                  int64
studio                  bool
total_area           float64
price                  int64
building_id            int64
build_year             int64
building_type_int      int64
latitude             float64
longitude            float64
ceiling_height       float64
flats_count            int64
floors_total           int64
has_elevator            bool
dtype: object

In [32]:
data=buff.copy()

In [33]:
data=data.drop(columns='studio')

In [34]:
def remove_duplicates(data: pd.DataFrame):
        feature_cols = data.columns.drop("floor_id").tolist()
        is_duplicated_features = data.duplicated(subset=feature_cols, keep=False)
        data = data[~is_duplicated_features].reset_index(drop=True)
        return data

data = remove_duplicates(data)
data

KeyError: "['floor_id'] not found in axis"

In [35]:
def remove_low_price_rows(data: pd.DataFrame):
    filtered_dataset = data[data['price'] > 800000]
    return filtered_dataset

def remove_high_price_rows(data: pd.DataFrame):
    filtered_dataset = data[data['price'] < 500000000]
    return filtered_dataset

def remove_outliers_iqr(data: pd.DataFrame):
    filtered_df = data.copy()
    num_cols = data.select_dtypes(['float','int']).columns
    threshold = 1.5
    for column in num_cols:
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR

        filtered_df = filtered_df[
            (filtered_df[column] >= lower_bound)
            & (filtered_df[column] <= upper_bound)
        ]

    return filtered_df

# data = data.drop(columns="studio")
# data = remove_duplicates(data)
data = remove_low_price_rows(data)
data = remove_high_price_rows(data)
data = remove_outliers_iqr(data)

data

,id,floor,is_apartment,kitchen_area,living_area,rooms,total_area,price,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9,False,9.90,19.900000,1,35.099998,9500000,6220,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,7,False,0.00,16.600000,1,43.000000,13500000,18012,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,9,False,9.00,32.000000,2,56.000000,13500000,17821,2000,4,55.740040,37.761742,2.70,80,10,True
4,4,3,False,3.00,14.000000,1,24.000000,5200000,9293,1971,1,55.808807,37.707306,2.60,208,9,True
5,5,9,False,0.00,0.000000,2,51.009998,8490104,23964,2017,4,55.724728,37.743069,2.70,192,17,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141356,141356,8,False,6.00,42.000000,3,64.000000,10800000,9503,1971,4,55.740402,37.834579,2.64,428,9,True
141358,141358,5,False,5.28,28.330000,2,41.110001,7400000,3162,1960,1,55.727470,37.768677,2.48,80,5,False
141359,141359,7,False,5.30,20.000000,1,31.500000,9700000,6513,1966,4,55.704315,37.506584,2.64,72,9,True
141360,141360,15,False,13.80,33.700001,2,65.300003,11750000,23952,2017,4,55.699863,37.939564,2.70,480,25,True


In [36]:
display(data['floor'].describe())

count    110215.000000
mean          6.728503
std           4.516316
min           1.000000
25%           3.000000
50%           6.000000
75%          10.000000
max          20.000000
Name: floor, dtype: float64

In [37]:
data.dtypes

id                     int64
floor                  int64
is_apartment            bool
kitchen_area         float64
living_area          float64
rooms                  int64
total_area           float64
price                  int64
building_id            int64
build_year             int64
building_type_int      int64
latitude             float64
longitude            float64
ceiling_height       float64
flats_count            int64
floors_total           int64
has_elevator            bool
dtype: object